# Interactive visualization of a robot in pydrake

*Copyright (c) 2018, Toyota Research Institute
All rights reserved.*

See license in `LICENSE`.

## Introduction
This notebook shows how to use Drake and [ipywidgets](https://github.com/jupyter-widgets/ipywidgets) to create an interactive visualization of a robot.

In [ ]:
import os.path
from pydrake.common import FindResourceOrThrow
from pydrake.multibody.rigid_body_tree import (
    AddModelInstanceFromUrdfStringSearchingInRosPackages,
    FloatingBaseType,
    RigidBodyTree,
)
from pydrake.lcm import DrakeLcm
from pydrake.systems.framework import BasicVector
from pydrake.multibody.rigid_body_plant import DrakeVisualizer
import numpy as np
import ipywidgets as widgets

## Convenience class for interactive visualization
The following class wraps a `DrakeVisualizer` system and connects it to Jupyter widgets.

In [ ]:
class DrakeVisualizerHelper:
    def __init__(self, tree):
        lcm = DrakeLcm()
        self.tree = tree
        self.visualizer = DrakeVisualizer(tree=self.tree, lcm=lcm, enable_playback=True)
        self.x = np.concatenate([robot.getZeroConfiguration(), 
                                 np.zeros(tree.get_num_velocities())])
        self.visualizer.PublishLoadRobot()
        self.draw(self.tree.getZeroConfiguration())
    def draw(self, q = None):
        if q is not None:
            self.x[:self.tree.get_num_positions()] = q
        context = self.visualizer.CreateDefaultContext()
        context.FixInputPort(0, BasicVector(self.x))
        self.visualizer.Publish(context)
        
    def inspect(self, slider_scaling = 1):
        # Setup widgets
        for i in range(robot.number_of_positions()):
            widgets.interact(
                self.__slider_callback,
                slider_value = widgets.FloatSlider(
                    value=slider_scaling * self.x[i],
                    min=slider_scaling * self.tree.joint_limit_min[i],
                    max=slider_scaling * self.tree.joint_limit_max[i],
                    description=self.tree.get_position_name(i)
                ),
                index=widgets.fixed(i),
                slider_scaling=widgets.fixed(slider_scaling)
            )

    def __slider_callback(self, slider_value, index, slider_scaling):
        self.x[index] = slider_value / slider_scaling
        self.draw()

## Demo

Instantiate a model of the IIWA robot arm along with a visualization helper.

In [ ]:
iiwa_file = FindResourceOrThrow("drake/manipulation/models/iiwa_description/urdf/iiwa14_spheres_collision.urdf")
robot = RigidBodyTree(iiwa_file, FloatingBaseType.kFixed)
#add_fixed_model(robot, iiwa_file)
vis_helper = DrakeVisualizerHelper(robot)

Draw the arm in a random configuration.

In [ ]:
q = robot.getRandomConfiguration()
vis_helper.draw(q)

Show the interactive sliders (with positions scaled to be in degrees).

In [ ]:
vis_helper.inspect(np.rad2deg(1))